# Loading & getting `MFLike` likelihood from `cobaya`

In this first tutorial, we will see how to instantiate `mflike` from `cobaya` interface and how we can retrieve the likelihood. This notebook is currently using the following modules: 

In [ ]:
import camb
import cobaya
import matplotlib as mpl
import numpy as np

print("     Numpy :", np.__version__)
print("Matplotlib :", mpl.__version__)
print("      CAMB :", camb.__version__)
print("    Cobaya :", cobaya.__version__)

## Installing `MFLike` likelihood data

We first start by declaring the likelihood within a `dict` object. Most of the likelihood's options are set by default. Here we only set the input `sacc` file and the associated covariance file.

In [ ]:
mflike_input_file = dict(
    input_file="LAT_simu_sacc_00044.fits", cov_Bbl_file="data_sacc_w_covar_and_Bbl.fits"
)
mflike_config = {"mflike.TTTEEE": mflike_input_file}

These files do not come with the `mflike` likelihood and we need to download and to install data needed by `mflike`. Here we will do it programatically in such a way we can use this notebook without any prerequisites. Code and data will be stored in the `/tmp` directory for the tutorial purpose but feel free to change it to whatever location you want. By the way the next command will take some times to proceed.

In [ ]:
import os
import tempfile

from cobaya.install import install
from cobaya.tools import resolve_packages_path

custom_packages_path = None

# Try to resolve global path
if custom_packages_path is None:
    packages_path = resolve_packages_path()
else:
    packages_path = custom_packages_path

install(
    {"likelihood": mflike_config},
    path=packages_path,
    no_set_global=True,
    skip_global=True,
)

## Loading `MFLike` *via* `cobaya`

Given a successful installation of the data, we then need to set the default values of CMB, foreground and nuisance parameters.

In [ ]:
from common import cosmo_params, fg_params, nuisance_params

We finally put everything into a dictionary to push it into `cobaya` configuration system. Here for the purpose of this tutorial, we use `camb` as Boltzmann solver with minimal settings. Another configuration `high_accuracy_settings` is also provided and you can switch within the `info` object.  
The likelihood also depends on the foreground model, the default model for which can also be obtained from mflike.BandpowerForeground.

In [ ]:
minimal_settings = dict(lens_potential_accuracy=1)

high_accuracy_settings = dict(
    lens_potential_accuracy=8,
    lens_margin=2050,
    AccuracyBoost=2.0,
    lSampleBoost=2.0,
    lAccuracyBoost=2.0,
    kmax=10,
    k_per_logint=130,
    nonlinear=True,
    DoLateRadTruncation=False,
)

info = {
    "params": cosmo_params | fg_params | nuisance_params,
    "likelihood": mflike_config,
    "theory": {
        "camb": {"extra_args": minimal_settings},
        "mflike.BandpowerForeground": None,
    },
    "packages_path": packages_path,
}

We will now get the model given the above parameters



In [ ]:
from cobaya.model import get_model

model = get_model(info)

To get a direct acces to the `MFLike` likelihood instance, we can retrieve it from the collection of
likelihoods of `cobaya` given its name (so far there is only our likelihood)



In [ ]:
mflike = model.likelihood["mflike.TTTEEE"]

We can now play with the different data (Bbl, inverted covariance matrix) and we can also call the different likelihood methods.